<H3>
  Preparation
</H3>

In [ ]:
#imports
import os
import numpy as np
import pandas as pd
import tensorflow as tf

import keras
from keras import applications, optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#set directories
#http://www.dataperspective.info/2019/02/how-to-import-data-into-google-colab.html
indir = '/content/drive/My Drive/School/DL/Assignments/Assignment2/flower_photos/'
outdir = '/content/drive/My Drive/School/DL/Assignments/Assignment2/models/'

source data for this assignment has been split at the first level into 75% training data, 25% test data. 20% of the training data has been held aside as a validation set, so the train-validation-test split of the original dataset is 60-15-25

In [ ]:
#define train image generator
#https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720
#https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
batch_size = 32

train_data_gen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True)

train_generator = train_data_gen.flow_from_directory(directory=indir + 'train',
                                                     target_size=(150, 150),
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='categorical',
                                                     shuffle=True,
                                                     seed=19)


Found 2201 images belonging to 5 classes.


In [ ]:
#define validation image generator
test_data_gen = ImageDataGenerator(rescale=1./255)

val_generator = test_data_gen.flow_from_directory(directory=indir + 'validation',
                                                  target_size=(150, 150),
                                                  color_mode='rgb',
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  seed=19)


Found 551 images belonging to 5 classes.


In [ ]:
#define the test image generator
test_generator = test_data_gen.flow_from_directory(directory=indir + 'test',
                                                   target_size=(150, 150),
                                                   color_mode='rgb',
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=True,
                                                   seed=19)

Found 918 images belonging to 5 classes.


In [ ]:
#load the pre trained VGG16 model
vgg = applications.VGG16(weights='imagenet',
                         include_top=False,
                         input_shape=(150, 150, 3))

In [ ]:
#print a summary of the transferred model
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

<H3>
  Model 1
</H3>

In [ ]:
#freeze all tranferred layers
for layer in vgg.layers:
  layer.trainable=False

In [ ]:
#confirm that layers have been frozen
for i, layer in enumerate(vgg.layers):
  print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False


In [ ]:
#create a new model with its own dense layers
model_1= Sequential([
    vgg,
    Flatten(name='new_flat_m1'),
    Dense(256, activation='relu', name='new_dense_m1'),
    Dropout(0.5),
    Dense(5, activation='softmax', name='new_pred_m1')
    
])



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
#print model_1 summary
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
new_flat_m1 (Flatten)        (None, 8192)              0         
_________________________________________________________________
new_dense_m1 (Dense)         (None, 256)               2097408   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
new_pred_m1 (Dense)          (None, 5)                 1285      
Total params: 16,813,381
Trainable params: 2,098,693
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
#compile the modle
model_1.compile(loss='categorical_crossentropy',
                optimizer=optimizers.SGD(lr=0.001, momentum=0.5),
                metrics=['accuracy'])

In [ ]:
#add callbacks
callbacks_1 = [EarlyStopping(monitor='val_acc', patience=2),
               ModelCheckpoint(filepath=os.path.join(outdir + 'model_1.h5'),
                               monitor='val_acc', 
                               save_best_only=True,
                               mode='auto')]

In [ ]:
#fit the model
model_1.fit_generator(train_generator,
                      steps_per_epoch=2201 // batch_size,
                      epochs=10,
                      callbacks=callbacks_1,
                      validation_data=val_generator,
                      validation_steps=551 // batch_size)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
68/68 [==============================] - 595s 9s/step - loss: 1.5612 - acc: 0.3332 - val_loss: 1.3389 - val_acc: 0.5294
Epoch 2/10
68/68 [==============================] - 589s 9s/step - loss: 1.3068 - acc: 0.4821 - val_loss: 1.2240 - val_acc: 0.5800
Epoch 3/10
68/68 [==============================] - 587s 9s/step - loss: 1.1910 - acc: 0.5514 - val_loss: 1.1297 - val_acc: 0.6204
Epoch 4/10
68/68 [==============================] - 587s 9s/step - loss: 1.0945 - acc: 0.5898 - val_loss: 1.0297 - val_acc: 0.6301
Epoch 5/10
68/68 [==============================] - 587s 9s/step - loss: 1.0574 - acc: 0.6079 - val_loss: 0.9871 - val_acc: 0.6590
Epoch 6/10
68/68 [==============================] - 587s 9s/step - loss: 0.9934 - acc: 0.6275 - val_loss: 0.9468 - val_acc: 0.7013
Epoch 7/10
68/68 [==============================] - 587s 9s/step - loss: 0.9401 - acc: 0.6545 - val_loss: 0.8426 - val

In [ ]:
#check test accuracy
loss_1, acc_1 = model_1.evaluate_generator(test_generator)

In [ ]:
#print model 1 accuracy
print('The accuracy for model 1 is: {}'.format(acc_1))


The accuracy for model 1 is: 0.7494553371712014


<H3>
  Model 2
</H3>

In [ ]:
#unfreeze block5 layers
for layer in vgg.layers:
  if layer.name[5:6]=='5':
    layer.trainable=True
  else:
    layer.trainable=False

In [ ]:
#confirm that layers have been unfrozen
for i, layer in enumerate(vgg.layers):
  print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True


In [ ]:
#create a new model with its own dense layers
model_2= Sequential([
    vgg,
    Flatten(name='new_flat_m2'),
    Dense(256, activation='relu', name='new_dense_m2'),
    Dropout(0.5),
    Dense(5, activation='softmax', name='new_pred_m2')
    
])


In [ ]:
#compile the modle
model_2.compile(loss='categorical_crossentropy',
                optimizer=optimizers.SGD(lr=0.001, momentum=0.5),
                metrics=['accuracy'])

In [ ]:
#add callbacks
callbacks_2 = [EarlyStopping(monitor='val_acc', patience=2),
               ModelCheckpoint(filepath=os.path.join(outdir + 'model_2.h5'),
                               monitor='val_acc', 
                               save_best_only=True,
                               mode='auto')]

In [ ]:
#fit the model
model_2.fit_generator(train_generator,
                      steps_per_epoch=2201 // batch_size,
                      epochs=5,
                      callbacks=callbacks_2,
                      validation_data=val_generator,
                      validation_steps=551 // batch_size)


Epoch 1/5
68/68 [==============================] - 692s 10s/step - loss: 1.4638 - acc: 0.3894 - val_loss: 1.0639 - val_acc: 0.6287
Epoch 2/5
68/68 [==============================] - 684s 10s/step - loss: 0.9959 - acc: 0.6146 - val_loss: 0.7504 - val_acc: 0.7437
Epoch 3/5
68/68 [==============================] - 682s 10s/step - loss: 0.7654 - acc: 0.7157 - val_loss: 0.6290 - val_acc: 0.7707
Epoch 4/5
68/68 [==============================] - 694s 10s/step - loss: 0.6573 - acc: 0.7656 - val_loss: 0.5908 - val_acc: 0.7803
Epoch 5/5
68/68 [==============================] - 688s 10s/step - loss: 0.5846 - acc: 0.7869 - val_loss: 0.5238 - val_acc: 0.8170


In [ ]:
#check test accuracy
loss_2, acc_2 = model_2.evaluate_generator(test_generator)

In [ ]:
#print model 2 accuracy
print('The accuracy for model 2 is: {}'.format(acc_2))

The accuracy for model 2 is: 0.8159041393036935


<H3>
  Model 3
</H3>

In [ ]:
#unfreeze block5 layers
for layer in vgg.layers:
  layer.trainable=True

In [ ]:
#confirm that all layers have been unfrozen
for i, layer in enumerate(vgg.layers):
  print(i, layer.name, layer.trainable)

0 input_1 True
1 block1_conv1 True
2 block1_conv2 True
3 block1_pool True
4 block2_conv1 True
5 block2_conv2 True
6 block2_pool True
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_pool True
11 block4_conv1 True
12 block4_conv2 True
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True


In [ ]:
#create a new model with its own dense layers
model_3= Sequential([
    vgg,
    Flatten(name='new_flat_m3'),
    Dense(256, activation='relu', name='new_dense_m3'),
    Dropout(0.5),
    Dense(5, activation='softmax', name='new_pred_m3')
    
])


In [ ]:
#compile the modle
model_3.compile(loss='categorical_crossentropy',
                optimizer=optimizers.SGD(lr=0.001, momentum=0.5),
                metrics=['accuracy'])

In [ ]:
#add callbacks
callbacks_3 = [EarlyStopping(monitor='val_acc', patience=2),
               ModelCheckpoint(filepath=os.path.join(outdir + 'model_3.h5'),
                               monitor='val_acc', 
                               save_best_only=True,
                               mode='auto')]

In [ ]:
#fit the model
model_3.fit_generator(train_generator,
                      steps_per_epoch=2201 // batch_size,
                      epochs=5,
                      callbacks=callbacks_3,
                      validation_data=val_generator,
                      validation_steps=551 // batch_size)


Epoch 1/5
68/68 [==============================] - 2010s 30s/step - loss: 1.0167 - acc: 0.6045 - val_loss: 0.5983 - val_acc: 0.7849
Epoch 2/5
68/68 [==============================] - 1968s 29s/step - loss: 0.5867 - acc: 0.7882 - val_loss: 0.5545 - val_acc: 0.7900
Epoch 3/5
68/68 [==============================] - 1914s 28s/step - loss: 0.4765 - acc: 0.8220 - val_loss: 0.5073 - val_acc: 0.8247
Epoch 4/5
68/68 [==============================] - 1907s 28s/step - loss: 0.4182 - acc: 0.8465 - val_loss: 0.3750 - val_acc: 0.8651
Epoch 5/5
68/68 [==============================] - 1897s 28s/step - loss: 0.3841 - acc: 0.8600 - val_loss: 0.3801 - val_acc: 0.8767


In [ ]:
#check test accuracy
loss_3, acc_3 = model_3.evaluate_generator(test_generator)

In [ ]:
#print model 3 accuracy
print('The accuracy for model 3 is: {}'.format(acc_3))

The accuracy for model 3 is: 0.8387799570763034


<H3>
  Conclusion
</H3>


Of the 3 models created here, the last performed the best (all layers unfrozen). This was the expected results because the ability to train more layers allows the model to converge better for  the task at hand. The risk in this approach is that the model might overfit the data since it is updating more weights.

There was some sign of overfitting in the third model, since the test accuracy was lower than both the validation and training accuracy. However, in spite of this, it was still broadly similar.
